# Multipair breakout optimiser

- Hunt 4h breakouts
- Indicator dependency resolution

# Set up

Set up Trading Strategy data client.

In [ ]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

client = Client.create_jupyter_client()

# Set up drawing charts in interactive vector output mode.
# This is slower. See the alternative commented option below.
# setup_charting_and_output(OutputMode.interactive)

# Set up rendering static PNG images.
# This is much faster but disables zoom on any chart.
setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)

# Parameters

- Strategy parameters define the fixed and grid searched parameters

In [ ]:
from tradingstrategy.chain import ChainId
import datetime

from skopt.space import Integer, Real, Categorical

from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.parameters import StrategyParameters


class Parameters:
    """Parameteres for this strategy.

    - Collect parameters used for this strategy here

    - Both live trading and backtesting parameters
    """

    id = "multipair-breakout-4h-optimiser-large" # Used in cache paths

    cycle_duration = CycleDuration.cycle_1h 
    candle_time_bucket = TimeBucket.h1
    allocation = 0.98   

    atr_length = Integer(10, 90)   # Bars
    fract = Real(0.2, 7)

    # Obtained in regime filter optimiser notebook series
    adx_length = 60
    adx_filter_threshold = 4.8
    
    trailing_stop_loss_pct = Real(0.95, 0.99)
    trailing_stop_loss_activation_level = Real(1.00, 1.08)
    stop_loss_pct = Real(0.95, 0.99)

    #
    # Live trading only
    #
    chain_id = ChainId.polygon
    routing = TradeRouting.default  # Pick default routes for trade execution
    required_history_period = datetime.timedelta(hours=200)

    #
    # Backtesting only
    #
    backtest_start = datetime.datetime(2018, 1, 1)
    backtest_end = datetime.datetime(2024, 6, 1)
    stop_loss_time_bucket = TimeBucket.m15
    backtest_trading_fee = 0.0005  # Override the default Binance data trading fee and assume we can trade 5 BPS fee on WMATIC-USDC on Polygon on Uniswap v3
    initial_cash = 10_000

parameters = StrategyParameters.from_class(Parameters, grid_search=True) 


# Trading pairs and market data

- Set up our trading pairs
- Load historical market data for backtesting
- We use Binance CEX data so we have longer history to backtest

In [ ]:
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data
from tradingstrategy.client import Client
from tradeexecutor.strategy.execution_context import ExecutionContext, notebook_execution_context
from tradeexecutor.utils.binance import create_binance_universe
from tradeexecutor.strategy.universe_model import UniverseOptions

# List of trading pairs we use in the backtest
# In this backtest, we use Binance data as it has more price history than DEXes
trading_pairs = [
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
    (ChainId.centralised_exchange, "binance", "MATIC", "USDT"),
    (ChainId.centralised_exchange, "binance", "LINK", "USDT"),
    (ChainId.centralised_exchange, "binance", "PEPE", "USDT"),
    (ChainId.centralised_exchange, "binance", "BNB", "USDT"),
]

def create_trading_universe(
    timestamp: datetime.datetime,
    client: Client,
    execution_context: ExecutionContext,
    universe_options: UniverseOptions,
) -> TradingStrategyUniverse:
    """Create the trading universe.

    - For live trading, we load DEX data

    - We backtest with Binance data, as it has more history
    """

    # Backtesting - load Binance data
    strategy_universe = create_binance_universe(
        [f"{p[2]}{p[3]}" for p in trading_pairs],
        candle_time_bucket=Parameters.candle_time_bucket,
        stop_loss_time_bucket=Parameters.stop_loss_time_bucket,
        start_at=universe_options.start_at,
        end_at=universe_options.end_at,
        trading_fee_override=Parameters.backtest_trading_fee,
        forward_fill=True,
    )
    return strategy_universe


strategy_universe = create_trading_universe(
    None,
    client,
    notebook_execution_context,
    UniverseOptions.from_strategy_parameters_class(Parameters, notebook_execution_context)
)

# Indicators

- We use `pandas_ta` Python package to calculate technical indicators
- These indicators are precalculated and cached on the disk
- Indicators are calculated to each pair in our trading pair dataset

In [ ]:
from tradeexecutor.state.identifier import TradingPairIdentifier
import pandas as pd
import pandas_ta

from tradeexecutor.analysis.regime import Regime
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource, IndicatorDependencyResolver
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradingstrategy.utils.groupeduniverse import resample_candles


def regime(
    close: pd.Series,
    adx_length: int,
    regime_threshold: float,
    pair: TradingPairIdentifier,
    dependency_resolver: IndicatorDependencyResolver,
) -> pd.Series:
    """A regime filter based on ADX indicator.

    Get the trend of BTC applying ADX on a daily frame.
    
    - -1 is bear
    - 0 is sideways
    - +1 is bull
    """
    adx_df = dependency_resolver.get_indicator_data(
        "adx",
        pair=pair,
        parameters={"length": adx_length},
        column="all",
    )

    adx_df = adx_df.shift(1)  # Look ahead bias trick

    # def regime_filter(row):
    #     # ADX, DMP, # DMN
    #     average_direction_index, directional_momentum_positive, directional_momentum_negative = row.values
    #     if directional_momentum_positive > regime_threshold:
    #         return Regime.bull.value
    #     elif directional_momentum_negative > regime_threshold:
    #         return Regime.bear.value
    #     else:
    #         return Regime.crab.value
    # regime_signal = adx_df.apply(regime_filter, axis="columns")    
    # return regime_signal

    def regime_filter(row):
        # ADX, DMP, # DMN
        average_direction_index, directional_momentum_positive, directional_momentum_negative = row.values
        if average_direction_index > regime_threshold:
            if directional_momentum_positive > directional_momentum_negative:
                return Regime.bull.value
            else:
                return Regime.bear.value
        else:
            return Regime.crab.value
    regime_signal = adx_df.apply(regime_filter, axis="columns")    
    return regime_signal


def create_indicators(
    timestamp: datetime.datetime | None,
    parameters: StrategyParameters,
    strategy_universe: TradingStrategyUniverse,
    execution_context: ExecutionContext
):
    indicators = IndicatorSet()

    indicators.add(
        "atr",
        pandas_ta.atr,
        {"length": parameters.atr_length},
        IndicatorSource.ohlcv,
    )    

    indicators.add(
        "adx",
        pandas_ta.adx,
        {"length": parameters.adx_length},
        IndicatorSource.ohlcv,
        order=2,
    )

    # A regime filter to detect the trading pair bear/bull markets
    indicators.add(
        "regime",
        regime,
        {"adx_length": parameters.adx_length, "regime_threshold": parameters.adx_filter_threshold},
        IndicatorSource.close_price,
        order=3.
    )
        
    return indicators


# Trading algorithm

- Describe out trading strategy as code

In [ ]:
from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput


def decide_trades(
    input: StrategyInput,
) -> list[TradeExecution]:

    # 
    # Decidion cycle setup.
    # Read all variables we are going to use for the decisions.
    #
    parameters = input.parameters
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators = input.indicators
    strategy_universe = input.strategy_universe
    cash = position_manager.get_current_cash()

    trades = []

    # Enable trailing stop loss after we reach the profit taking level
    #
    for position in state.portfolio.open_positions.values():
        if position.trailing_stop_loss_pct is None:
            close_price = indicators.get_price(pair=position.pair)
            if close_price >= position.get_opening_price() * parameters.trailing_stop_loss_activation_level:
                position.trailing_stop_loss_pct = parameters.trailing_stop_loss_pct                

    # Trade each of our pair individually.
    #
    # If any pair has open position,
    # do not try to rebalance, but hold that position to the end.
    #
    for pair_desc in trading_pairs:
        pair = strategy_universe.get_pair_by_human_description(pair_desc)

        close_price = indicators.get_price(pair=pair)  # Price the previous 15m candle closed for this decision cycle timestamp
        atr = indicators.get_indicator_value("atr", pair=pair)  # The ATR value at the time of close price
        # last_hour = (timestamp - parameters.cycle_duration.to_pandas_timedelta()).floor(freq="H")  # POI (point of interest): Account 15m of lookahead bias whehn using decision cycle timestamp
        last_hour = (timestamp - parameters.cycle_duration.to_pandas_timedelta()).floor(freq="4H")  # POI (point of interest): Account 15m of lookahead bias whehn using decision cycle timestamp
        previous_price = indicators.get_price(pair=pair, timestamp=last_hour)  # The price at the start of the breakout period
        regime_val = indicators.get_indicator_value("regime", pair=pair, data_delay_tolerance=pd.Timedelta(hours=24))  # Because the regime filter is calculated only daily, we allow some lookback
    
        if None in (atr, close_price, previous_price):
            # Not enough historic data,
            # cannot make decisions yet
            continue
        
        # If regime filter does not have enough data at the start of the backtest,
        # default to bull market
        regime = Regime(regime_val)  # Convert to enum for readability
                
        # We assume a breakout if our current 15m candle has closed
        # above the 1h starting price + (atr * fraction) target level
        long_breakout_entry_level = previous_price + atr * parameters.fract

        # Check for open condition - is the price breaking out
        #
        if not position_manager.is_any_open():
            if regime == Regime.bull:  
                if close_price > long_breakout_entry_level:
                    trades += position_manager.open_spot(
                        pair,
                        value=cash * parameters.allocation,
                        stop_loss_pct=parameters.stop_loss_pct,             
                    )

    # Visualisations
    #
    if input.is_visualisation_enabled():
        visualisation = state.visualisation
        # Visualise ATR for BTC        
        pair = strategy_universe.get_pair_by_human_description(trading_pairs[0])
        visualisation.plot_indicator(
            timestamp, 
            "ATR", 
            PlotKind.technical_indicator_detached, 
            atr,
            pair=pair,
        )

    return trades  # Return the list of trades we made in this cycle

# Optimise

- Run the optimiser
- `iterations` will tell you how many optimisation rounds are performed
    - Start with a small value like 6 to see your code runs well and then increase to get more refined results
    - Inspect the resulting parameters to see if you are hitting the boundaries of your search space and you need to expand the range of searched parameters
    - If it looks like adding iterations won't find better results then you have likely find the global optimum
    - Each iteration will make the optimisation run linearly longer
    - Each iteration sends a batch of Gaussian Proces points for the child process fleet to crunch, equal to the number of GPU - so the amount of results will iterations * CPUs 
- Optimise for max CAGR (do not care about the risk)
    - Alternative you could optimise for Sharpe, or any custom 
- Optimiser will automatically use most available CPUs on your computer
- Optimiser has a timeout to detect hung/broken backtests that take forever to complete, see `perform_optimisation(timeout)` argument
    - The default timeout for a single backtest is 10 minutes
- `perform_optimisation(result_filter)` argument is set to filter out strategies that take only small number of trades - they are likely statistical flukes and pollute genuine results 

In [ ]:
import logging
from tradeexecutor.backtest.optimiser import perform_optimisation
from tradeexecutor.backtest.optimiser import prepare_optimiser_parameters
from tradeexecutor.backtest.optimiser import optimise_profit, optimise_sharpe
from tradeexecutor.backtest.optimiser import MinTradeCountFilter

# How many Gaussian Process iterations we do
iterations = 8

optimised_results = perform_optimisation(
    iterations=iterations,
    search_func=optimise_sharpe,
    decide_trades=decide_trades,
    strategy_universe=strategy_universe,
    parameters=prepare_optimiser_parameters(Parameters),  # Handle scikit-optimise search space
    create_indicators=create_indicators,
    result_filter=MinTradeCountFilter(150),
    timeout=20*60,    
    # Uncomment for diagnostics
    # log_level=logging.INFO,
    # max_workers=1,
)

print(f"Optimise completed, optimiser searched {optimised_results.get_combination_count()} combinations")

## Results

- Show the top results of all optimiser iterations in a single table

In [ ]:
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import render_grid_search_result_table

#: How many results we analyse from the top 
cut_off = 100

all_combinations = optimised_results.get_results_as_grid_search_results() 

# Optimiser already filtered for min_positions_threshold when doing the optimiser run
df = analyse_grid_search_result(all_combinations[0:cut_off], min_positions_threshold=0)
print(f"Showing the best {len(df)} results")

render_grid_search_result_table(df)


Render the candidate with the best equity curve.

In [ ]:
from tradeexecutor.visual.grid_search import visualise_single_grid_search_result_benchmark

fig = visualise_single_grid_search_result_benchmark(
    all_combinations[0], 
    strategy_universe, 
    initial_cash=Parameters.initial_cash
)
fig.show()

# Portfolio performance (best pick)

- Compare buy and hold against our best performance

In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import compare_strategy_backtest_to_multiple_assets

returns = all_combinations[0].returns

compare_strategy_backtest_to_multiple_assets(
    state=None,
    strategy_universe=strategy_universe,
    returns=returns,
    display=True,
)

# Trade summary (best pick)

- Show statistics about the made trades

In [ ]:
summary = all_combinations[0].summary
display(summary.to_dataframe())

# Trading pair performance breakdown

- Show breakdown of different pairs on the best result

In [ ]:
from tradeexecutor.analysis.multipair import analyse_multipair
from tradeexecutor.analysis.multipair import format_multipair_summary

state = all_combinations[0].hydrate_state()

multipair_summary = analyse_multipair(state)
display(format_multipair_summary(multipair_summary))